In [ ]:
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('datasets/dataset_stemming.csv')
df.head(10)

,tweet,label
0,jln jatibarupolisi tdk bs gertak gubernur eman...,anger
1,cewe lho kayak rasain sibuk jaga rasain sakit ...,anger
2,kepingin gudeg mbarek bu hj amad foto google s...,happy
3,jln jatibarubagian wilayah tn abangpengaturan ...,anger
4,sharing alam aja kemarin jam 1800 batalin tike...,happy
5,sekian thread baca thread aneh sih tulis sumpa...,anger
6,sharing temen tuh emg bgt saat lu ngerasa lu b...,happy
7,orang pake ponco jas hujan pake kasur ya gara2...,sadness
8,contoh yg gemar sudut teriak toleran tp gemar ...,anger
9,pulang udah h-4 lebaran dilema apa2 rumah leba...,sadness


In [ ]:
df.label.unique()

array(['anger', 'happy', 'sadness', 'love', 'fear'], dtype=object)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
#preprocessing
print(df['tweet'].apply(lambda x: len(x.split(' '))).sum())

82064


In [ ]:
special_character_remover = re.compile('[/(){}\[\]\|@,;]')
extra_symbol_remover = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
    text = text.lower()
    text = special_character_remover.sub(' ', text)
    text = extra_symbol_remover.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
    
df['tweet'] = df['tweet'].apply(clean_text)

In [ ]:
print(df['tweet'].apply(lambda x: len(x.split(' '))).sum())

78837


In [ ]:
from sklearn.model_selection import train_test_split
X = df.tweet
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape


((3501,), (876,), (3501,), (876,))

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])

lr.fit(X_train,y_train)
y_pred1 = lr.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred1,y_test)}")

Accuracy is : 0.6541095890410958


In [ ]:
from sklearn.naive_bayes import MultinomialNB


naivebayes = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
naivebayes.fit(X_train, y_train)

y_pred = naivebayes.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

accuracy 0.5958904109589042


In [ ]:
from xgboost import XGBClassifier

xgboost = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgboost.fit(X_train, y_train)

y_pred = xgboost.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

C:\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:59:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.6015981735159818


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier()),
              ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

accuracy 0.6221461187214612
